In [1]:
import numpy as np
import time
import cv2
from boosting_classifier import Boosting_Classifier
from visualizer import Visualizer
from im_process import normalize
from utils import *

In [2]:
flag_subset = True
boosting_type = 'Ada' #'Real' or 'Ada'
training_epochs = 100 if not flag_subset else 20
act_cache_dir = 'wc_activations.npy' if not flag_subset else 'wc_activations_subset.npy'
chosen_wc_cache_dir = 'chosen_wcs.pkl' if not flag_subset else 'chosen_wcs_subset.pkl'

In [3]:
#data configurations
pos_data_dir = '../newface16'
neg_data_dir = '../nonface16'
image_w = 16
image_h = 16
data, labels = load_data(pos_data_dir, neg_data_dir, image_w, image_h, flag_subset)
data = integrate_images(normalize(data))

Load in 200 images, 100 faces, 100 non-faces


In [4]:
#number of bins for boosting
num_bins = 25

#number of cpus for parallel computing
num_cores = 8 if not flag_subset else 1 #always use 1 when debugging

In [5]:
#create Haar filters
filters = generate_Haar_filters(4, 4, 16, 16, image_w, image_h, flag_subset)

In [6]:
#create visualizer to draw histograms, roc curves and best weak classifier accuracies
drawer = Visualizer([10, 20, 50, 100], [1, 10, 20, 50, 100])

In [7]:
#create boost classifier with a pool of weak classifier
boost = Boosting_Classifier(filters, data, labels, training_epochs, num_bins, drawer, num_cores, boosting_type)

In [8]:
#calculate filter values for all training images
start = time.clock()
boost.calculate_training_activations(act_cache_dir, act_cache_dir)
end = time.clock()
print('%f seconds for activation calculation' % (end - start))

Calcuate activations for 1000 weak classifiers, using 200 imags.
[Find cached activations, wc_activations_subset.npy loading...]
0.002070 seconds for activation calculation


# Code Testing

In [9]:
#weight initialization
weights = [1/200.0]*200

## Single Weak Classifier - Adaboost WC Object

In [10]:
wc1 = boost.weak_classifiers[0]

In [11]:
print(wc1.polarity)
print(wc1.threshold)

None
None


In [12]:
min_error = wc1.calc_error(weights, labels)

In [13]:
min_error

0.44

In [14]:
print(wc1.polarity)
print(wc1.threshold)

1
-0.1414965986394563


## Multiple weak classifiers as attributes of Boosting Classifier object

In [15]:
print(boost.weak_classifiers[0].polarity)
print(boost.weak_classifiers[0].threshold)

print(boost.weak_classifiers[1].polarity)
print(boost.weak_classifiers[1].threshold)

1
-0.1414965986394563
None
None


In [16]:
classifier_errors = boost.weak_classifier_errors(weights)

In [17]:
print(boost.weak_classifiers[0].polarity)
print(boost.weak_classifiers[0].threshold)

print(boost.weak_classifiers[1].polarity)
print(boost.weak_classifiers[1].threshold)

1
-0.1414965986394563
-1
0.1607843137254905


In [18]:
print(classifier_errors[0])
print(classifier_errors[1])

0.44
0.43999999999999995


In [28]:
min_err, best_wc_indx = boost.select_best_weak_classifier(classifier_errors)
print(min_err, best_wc_indx)

0.20499999999999996 223


In [30]:
alph = boost.calculate_alpha(min_err)
print(alph)

1.355332135515924


In [32]:
updated_weights = boost.update_weights(boost.weak_classifiers[best_wc_indx], weights, alph)
print(updated_weights[0:10])

[0.01939024390243903, 0.01939024390243903, 0.01939024390243903, 0.01939024390243903, 0.01939024390243903, 0.01939024390243903, 0.01939024390243903, 0.01939024390243903, 0.01939024390243903, 0.01939024390243903]


In [33]:
print(np.array(weights[0:10]) -np.array(updated_weights[0:10]))

[-0.01439024 -0.01439024 -0.01439024 -0.01439024 -0.01439024 -0.01439024
 -0.01439024 -0.01439024 -0.01439024 -0.01439024]


In [39]:
[(classifier_errors[i], boost.weak_classifiers[i].threshold, boost.weak_classifiers[i].polarity)  for i in range(10)]

[(0.44, 0.23321328531412444, -1),
 (0.43999999999999995, 0.1607843137254905, -1),
 (0.365, 0.19263705482192872, -1),
 (0.365, 0.12941176470588012, -1),
 (0.365, 0.26562625050019983, -1),
 (0.41999999999999993, -0.36110444177671197, -1),
 (0.46499999999999997, 0.04481792717086819, -1),
 (0.445, -0.3816726690676244, 1),
 (0.42, -0.25282112845137616, 1),
 (0.455, -0.06634653861544892, 1)]

In [ ]:
chosenWeakClassifiers = boost.train(chosen_wc_cache_dir)

In [ ]:
#boost.visualize()

In [ ]:
#original_img = cv2.imread('../Testing_Images/Face_1.jpg', cv2.IMREAD_GRAYSCALE)
#result_img = boost.face_detection(original_img)
#cv2.imwrite('Result_img_%s.png' % boosting_type, result_img)